In [26]:
# Import each module in the order that it will be executed
import PySAM.Pvsamv1 as PVSAM
import PySAM.Grid as Grid
import PySAM.Battery as Battery
import PySAM.Windpower as Windpower
import PySAM.Utilityrate5 as UtilityRate
import PySAM.Singleowner as SingleOwner
# import PySAM.Hybrids as Hybrids

# Also import JSON to load the inputs
import json
# For paths
import os
import pandas as pd
import pathlib

In [2]:
pv_model = PVSAM.new()
battery = Battery.from_existing(pv_model)
# windpower = Windpower.from_existing(pv_model)
grid = Grid.from_existing(pv_model)
utility_rate = UtilityRate.from_existing(pv_model)
single_owner = SingleOwner.from_existing(pv_model)

# %% Load the inputs from the JSON file for each module
dir = 'data/PySAM_Inputs/Minimum Viable System/'
prefix = 'project2_'
file_names = ["pvsamv1", "battery", "grid", "utilityrate5", "singleowner"]
modules = [pv_model, battery, grid, utility_rate, single_owner]
for f, m in zip(file_names, modules):
    filepath = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2', "data", "PySam_Inputs", "Minimum Viable System", prefix + f + '.json')
    print(f"Loading inputs from {filepath}")
    with open(filepath, 'r') as file:
        data = json.load(file)
        # Loop through each key-value pair and set the module inputs
        for k, v in data.items():
            # Note: I'm ignoring any 'adjustment factors' here, but these can be set afterwards.
            # See: https://nrel-pysam.readthedocs.io/en/main/modules/Pvsamv1.html#adjustmentfactors-group
            if k != 'number_inputs' and 'adjust_' not in k:
                try:
                    m.value(k, v)
                except:
                    continue

Loading inputs from /home/emco4286/ecen5407/ecen5407_project2/data/PySam_Inputs/Minimum Viable System/project2_pvsamv1.json
Loading inputs from /home/emco4286/ecen5407/ecen5407_project2/data/PySam_Inputs/Minimum Viable System/project2_battery.json
Loading inputs from /home/emco4286/ecen5407/ecen5407_project2/data/PySam_Inputs/Minimum Viable System/project2_grid.json
Loading inputs from /home/emco4286/ecen5407/ecen5407_project2/data/PySam_Inputs/Minimum Viable System/project2_utilityrate5.json
Loading inputs from /home/emco4286/ecen5407/ecen5407_project2/data/PySam_Inputs/Minimum Viable System/project2_singleowner.json


In [3]:
filename = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2', "data", "222628_32.73_-117.18_2012_interpolated.csv")
pv_model.SolarResource.solar_resource_file = filename
pv_model.BatterySystem.batt_replacement_option = 0

In [4]:
modules = [pv_model, battery, grid, utility_rate, single_owner]
for m in modules:
    m.execute()

In [ ]:
import pandas as pd

In [9]:
nsrdb = pd.read_csv(filename, skiprows=2, usecols=['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'DHI', 'GHI',
       'DNI', 'Surface Albedo', 'Wind Speed', 'Pressure'])
local_index = pd.DatetimeIndex(nsrdb['Year'].astype(str) + '-' + nsrdb['Month'].astype(str) +
                             '-' + nsrdb['Day'].astype(str) + ' ' + nsrdb['Hour'].astype(str) +
                               ':' + nsrdb['Minute'].astype(str))

In [21]:
leap_day = pd.date_range(start='2012-02-29 00:00:00',end='2012-02-29 23:55:00', freq='5min')
new_index = local_index.drop(leap_day)

In [22]:
generation = pd.Series(data=list(pv_model.Outputs.gen), index=new_index)

In [24]:

save_pt = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2', "data", "sienna_pv_gen.csv")
generation.to_csv(save_pt)

In [25]:
# nsrdb['ac_gross'] = list(pv_model.Outputs.ac_gross)[:105408]
# nsrdb['dc_net'] = list(pv_model.Outputs.dc_net)[:105408]

# nsrdb['batt_capacity_percent'] = list(pv_model.Outputs.batt_capacity_percent)[:105408]
# nsrdb["batt_power_dc"] = list(pv_model.Outputs.batt_power_dc)[:105408]
# nsrdb["batt_power"] = list(pv_model.Outputs.batt_power)[:105408]
# nsrdb["gen"] = list(pv_model.Outputs.gen)[:105408]
# nsrdb['batt_SOC'] = list(pv_model.Outputs.batt_SOC)[:105408]

In [27]:
base_dir = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2')
data_pt = os.path.join(base_dir, "data", "Project 2 - Load Profile_modified_index2.csv")

load = pd.read_csv(data_pt)

In [28]:
load

,Datetime,Datetime.1,Load (MW)
0,2012-01-01 00:00:00,1/1/2012 0:00,0.354184
1,2012-01-01 00:05:00,1/1/2012 0:05,0.352521
2,2012-01-01 00:10:00,1/1/2012 0:10,0.351602
3,2012-01-01 00:15:00,1/1/2012 0:15,0.349985
4,2012-01-01 00:20:00,1/1/2012 0:20,0.349208
...,...,...,...
105403,2012-12-31 23:35:00,12/31/2012 23:35,0.395603
105404,2012-12-31 23:40:00,12/31/2012 23:40,0.393632
105405,2012-12-31 23:45:00,12/31/2012 23:45,0.391683
105406,2012-12-31 23:50:00,12/31/2012 23:50,0.389567


In [32]:
my_index = pd.DatetimeIndex(load['Datetime'])
load.set_index(my_index, inplace=True)

In [33]:
leap_day = pd.date_range(start='2012-02-29 00:00:00',end='2012-02-29 23:55:00', freq='5min')
new_index = my_index.drop(leap_day)

In [36]:
data = load.loc[new_index, 'Load (MW)'].values

In [37]:
electric_load = pd.Series(index=new_index, data=data)

In [38]:
save_pt = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2', "data", "sienna_load.csv")
electric_load.to_csv(save_pt)